In [1]:
import numpy as np

In [2]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
header = ["color", "diameter", "label"]

In [11]:
def get_unique_vals(array, col):
    return {v[col] for v in array}

In [15]:
get_unique_vals(training_data, 2)

{'Apple', 'Grape', 'Lemon'}

In [18]:
def class_counts(data):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in data:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [19]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [20]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [22]:
is_numeric(5)

True

In [55]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {header[self.column]} {condition} {str(self.value)}?"
            
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value



In [56]:
q = Question(0, "Green")

In [57]:
q.match(training_data[0])

True

In [58]:
training_data[0]

['Green', 3, 'Apple']

In [59]:
training_data[0][0]

'Green'

In [60]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows = list()
    false_rows = list()
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [61]:
tr, fr = partition(training_data, Question(0, 'Red'))

In [62]:
tr

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [63]:
fr

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [91]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    Returns the chance of missclassifying the label.
    
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [92]:
no_mixing = [['Apple'], ['Apple']]

In [93]:
gini(no_mixing)

0.0

In [94]:
some_mixing = [['Apple'], ['Orange'], ['Apple']]

In [95]:
gini(some_mixing)

0.4444444444444445

In [96]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [97]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [ ]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

In [81]:
true_rows

[['Green', 3, 'Apple']]

In [82]:
false_rows

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [84]:
info_gain(true_rows, false_rows, current_uncertainty)

{'Apple': 1}
{'Apple': 1, 'Grape': 2, 'Lemon': 1}


0.1399999999999999

In [90]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

{'Grape': 2}
{'Apple': 2, 'Lemon': 1}
0.37333333333333324
